#Disruptive Architectures - Challenge Sprint 3


## Integrantes do grupo

  | Nome                         | RM     |
  |------------------------------|--------|
  | Natan Junior Rodrigues Lopes | 552626 |
  | Pedro Lucca Medeiros Miranda | 553873 |
  | Pedro Moreira de Jesus       | 553912 |

## Contextualizando

Nesta terceira Sprint da matéria de Disruptive Architectures, nós desempenhamos o trabalho de análise de dados, necessário para a criação de uma versão mais robusta do nosso modelo de Inteligência Artificial, pronta para ser utilizada em uma API Rest desenvolvida com Flask.

## Instalando o Faker e importando as dependências

In [44]:
!pip install faker
import pandas as pd
import numpy as np
from faker import Faker
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

## Criando o dataset com dados falsos de teste

Para criar nosso dataset, fizemos uma análise de dados que possuem relação com o acontecimento do sinistro odontológico. Entendendo a relação desses fatores e seu impacto no _target_ sinistro, nós conseguimos montar nossa tabela utilizando o Pandas, Numpy e o Faker.

In [25]:
fake = Faker()
num_records = 10000

data = {
    'idade': np.random.randint(18, 80, size=num_records),
    'genero': np.random.choice(['M', 'F'], size=num_records),
    'frequencia_consultas': np.random.poisson(2, num_records),
    'aderencia_tratamento': np.random.uniform(0, 1, num_records),
    'historico_caries': np.random.binomial(1, 0.3, size=num_records),
    'doenca_periodontal': np.random.binomial(1, 0.2, size=num_records),
    'numero_implantes': np.random.randint(0, 5, size=num_records),
    'tratamentos_complexos_previos': np.random.binomial(1, 0.3, size=num_records),
    'fumante': np.random.binomial(1, 0.2, size=num_records),
    'alcoolismo': np.random.binomial(1, 0.1, size=num_records),
    'escovacao_diaria': np.random.choice([1, 2, 3], size=num_records, p=[0.2, 0.6, 0.2]),
    'uso_fio_dental': np.random.binomial(1, 0.5, size=num_records),
    'doencas_sistemicas': np.random.choice(['Diabetes', 'Nenhuma', 'Cardíaca', 'Outras'], size=num_records, p=[0.1, 0.7, 0.1, 0.1]),
    'medicamentos_uso_continuo': np.random.poisson(1, num_records),
    'numero_sinistros_previos': np.random.poisson(1, num_records),
    'tipo_plano': np.random.choice(['basico', 'intermediario', 'avancado'], size=num_records, p=[0.5, 0.3, 0.2]),
}

df = pd.DataFrame(data)

df['probabilidade_sinistro'] = 0.1

# Aqui, nós combinamos nossos dados para gerar a probabilidade de sinistro, nossa coluna target.
df['probabilidade_sinistro'] += df['idade'].apply(lambda x: 0.05 if x > 60 else 0)
df['probabilidade_sinistro'] += df['doenca_periodontal'] * 0.1
df['probabilidade_sinistro'] += df['historico_caries'] * 0.05
df['probabilidade_sinistro'] += df['fumante'] * 0.1
df['probabilidade_sinistro'] += df['alcoolismo'] * 0.05
df['probabilidade_sinistro'] += df['tratamentos_complexos_previos'] * 0.08
df['probabilidade_sinistro'] += df['numero_sinistros_previos'] * 0.05
df['probabilidade_sinistro'] += df['doencas_sistemicas'].apply(lambda x: 0.1 if x != 'Nenhuma' else 0)

# Já nesta etapa, reduzimos a probabilidade de sinistro com base em fatores
# como a prevenção de problemas com a saúde bucal e o tratamento de doenças
df['probabilidade_sinistro'] -= df['frequencia_consultas'] * 0.02
df['probabilidade_sinistro'] -= df['aderencia_tratamento'] * 0.1

df['probabilidade_sinistro'] = df['probabilidade_sinistro'].clip(0, 1)

## Dataset criado

Por fim, esta foi a estrutura do Dataset:

In [ ]:
df

,idade,genero,localizacao,frequencia_consultas,aderencia_tratamento,historico_caries,doenca_periodontal,numero_implantes,tratamentos_complexos_previos,fumante,alcoolismo,escovacao_diaria,uso_fio_dental,doencas_sistemicas,medicamentos_uso_continuo,numero_sinistros_previos,tipo_plano,probabilidade_sinistro
0,59,F,Lake Meaganside,3,0.814988,1,1,0,0,0,0,2,0,Nenhuma,2,1,intermediario,0.158501
1,49,F,Jakechester,1,0.047018,0,0,2,1,0,1,1,0,Nenhuma,0,2,avancado,0.305298
2,25,M,Nelsonmouth,6,0.057382,1,0,1,1,0,0,1,0,Nenhuma,1,2,intermediario,0.204262
3,23,M,Hendersonborough,2,0.590588,1,0,2,0,1,0,2,1,Diabetes,0,1,basico,0.300941
4,44,F,Millermouth,1,0.496114,0,0,1,1,0,0,2,1,Cardíaca,0,2,intermediario,0.310389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,64,F,South Andre,2,0.970265,1,1,0,1,0,1,3,0,Diabetes,0,2,basico,0.492973
9996,19,F,Lake Amyport,4,0.722158,1,0,1,0,0,0,2,1,Diabetes,1,1,basico,0.147784
9997,57,M,Port Mark,4,0.313524,0,0,1,0,0,0,3,1,Nenhuma,1,3,basico,0.138648
9998,26,F,Lisaview,2,0.232556,1,1,1,0,0,0,3,0,Nenhuma,2,1,intermediario,0.236744


##Treinamento do modelo

Em seguida, fizemos o tratamento e organização dos dados para iniciar o treinamento do modelo


Primeiro, utilizamos o Label Encoder para transformar nossos dados textuais e categóricos em numéricos

In [43]:
X = df.drop('probabilidade_sinistro', axis=1)
y = df['probabilidade_sinistro']

### Instanciando o GBR

Em seguida, dividimos o Dataset em treino e teste, com 20% de dados para teste e
por fim, nós instanciamos um modelo de regressão chamado Gradient Boosting Regressor, uma vez que ao trabalhar com valores percentuais, um modelo de regressão é o mais adequado. Além disso, devido a estrutura e baixa complexidade de nosso Dataset, Redes Neurais não seriam tão eficientes.

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

E então, testamos o modelo:

## Criação da Pipeline, exportação e teste final do modelo
Para garantir a qualidade do modelo, e utilizá-lo em uma API Flask, nós criamos uma Pipeline, que é um processo que segue uma série de passos que definimos antes de realizar a predição dos dados.

Criamos essa Pipeline com o intuito de abstrair a lógica de tratamento, Normalização e Encoding dos dados, para que uma vez que tivermos o modelo exportado, ele já tenha consigo embutido um processo responsável por preparar os dados para previsão.

Por fim, geramos dois dados de teste que passarão pela Pipeline, serão tratados e por fim terão seu risco de Sinistro previsto pelo nosso modelo.

In [41]:
data_to_predict = pd.DataFrame({
    'idade': [30, 65],
    'genero': ['M', 'F'],
    'frequencia_consultas': [3, 1],
    'aderencia_tratamento': [0.8, 0.5],
    'historico_caries': [0, 1],
    'doenca_periodontal': [1, 0],
    'numero_implantes': [0, 2],
    'tratamentos_complexos_previos': [0, 1],
    'fumante': [0, 1],
    'alcoolismo': [0, 0],
    'escovacao_diaria': [2, 1],
    'uso_fio_dental': [1, 0],
    'doencas_sistemicas': ['Nenhuma', 'Diabetes'],
    'medicamentos_uso_continuo': [0, 2],
    'numero_sinistros_previos': [0, 1],
    'valor_medio_sinistros': [500, 1500],
    'tipo_plano': ['intermediario', 'avancado'],
})

### Criação da Pipeline


In [40]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier

# Aqui, dividimos e listamos nossas features para mapeá-las na etapa
# de transformação da Pipeline
categorical_features = ['genero', 'tipo_plano', 'doencas_sistemicas']
numerical_features = [
    'idade', 'frequencia_consultas', 'aderencia_tratamento', 'historico_caries',
    'doenca_periodontal', 'numero_implantes', 'tratamentos_complexos_previos',
    'fumante', 'alcoolismo', 'escovacao_diaria', 'uso_fio_dental',
    'medicamentos_uso_continuo', 'numero_sinistros_previos'
]

# Nessa etapa criamos as transformações que o preprocessor fará
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numerical_features),
    ('cat', OneHotEncoder(), categorical_features)
])

# Por fim instanciamos a Pipeline com o pré-processamento e o modelo que escolhemos,
# Neste caso o GBR, com motivação para escolha explicada na Sprint anterior
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', GradientBoostingRegressor())
])

### Treinamento do modelo e exportação do modelo

In [45]:
# Treinando o modelo com a pipeline
pipeline.fit(X_train, y_train)

predictions = pipeline.predict(data_to_predict)
limiar = 0.4
classifications = ["Alto Risco" if prob > limiar else "Baixo Risco" for prob in predictions]
classifications

['Baixo Risco', 'Alto Risco']

In [38]:
y_pred = pipeline.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse:.4f}')
print(f'R²: {r2:.4f}')

MSE: 0.0001
R²: 0.9901


### Exportando o modelo em formato pickle, que será importado utilizando o joblib no nosso projeto Flask

In [46]:
import joblib
joblib.dump(pipeline, 'Aletheia.pkl')

['Aletheia.pkl']